In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image, ImageSequence, ImageFilter
import roifile
from roifile import ImagejRoi
import zipfile
import os
import time

To prepare for a run:
1. Define a data output path for the full dataframe to output data including viability, circle centers, and radius for each spheroid and each image in the outer loop
2. Set tiff paths 1 and 2 to the correct paths. This is split into two parts to allow the outer loop to cycle over several TIFFs
3. Repeat step 2 for the zipped folder of ROIs
4. Set the outer loop to the correct number of images
5. Set the correct threshold for the binarization of images

In [ ]:
start = time.time()

threshold = 30

full_df = pd.DataFrame()
data_output_path = "[Cell Line]/[Drug Concentration]/[Output].csv"
tiff_path_1 = "[Cell Line]/[Drug Concentration]/[Image Information 1] "
tiff_path_2 = "[Image Information 2].tif"
zip_path_1 = "[Cell Line]/[Drug Concentration]/[ROI] [Image Information 1] "
zip_path_2 = "[Image Information 2].zip"
plot_path = tiff_path_1

for num_str in ["1", "2", "3", "4"]:
    print("----------------------------")
    print("Running analysis for TIFF " + num_str)

    # Read in TIFF file
    im = Image.open(tiff_path_1 + num_str + tiff_path_2)

    # Create dictionary of individual images from multilayer TIFF
    img_dict = {}
    page_list = []
    for i, page in enumerate(ImageSequence.Iterator(im)):
        page.save("page%d.png" % i)
        img_dict[i] = "page%d.png" % i
        page_list.append(Image.open(img_dict[i]))

    # Remove any preexisting .roi files
    for file in os.listdir():
        if '.roi' in file:
            os.remove(file)

    # Unzip ROI folder
    with zipfile.ZipFile(zip_path_1 + num_str + zip_path_2) as zip_ref:
        zip_ref.extractall(".")

    # Create list of all ROIs present and their definitions (using roifile library)
    roi_list = []
    for file in os.listdir():
        if '.roi' in file:
            roi = ImagejRoi.fromfile(file)
            roi_list.append(roi)
    #print(len(roi_list))

    # For each ROI, run through each image slice
    viability_dict = {}
    Pixel_to_um = 1.3541432584269662921348314606742

    j = 0
    for roi in roi_list:
        counts_dict = {'red':[], 'blue':[], 'total':[]}
        left = roi.left
        right = roi.right
        top = roi.top
        bottom = roi.bottom
        # This radius is simply the roi scaffold radius
        radius = (right - left) / 2
        cx = left + radius
        cy = bottom + radius
        for i in range(int(len(img_dict))//3):
            r_idx = 3*i
            g_idx = 3*i + 1
            b_idx = 3*i + 2
            # Red channel (channel 1, dead cells)
            r_img = page_list[r_idx]
            r_img = r_img.point(lambda p: p > threshold and 255)
            # Green channel (channel 2)
            g_img = page_list[g_idx]
            # Blue channel (channel 3, live cells)
            # Binarize the image using [threshold]
            b_img = page_list[b_idx]
            b_img = b_img.point(lambda p: p > threshold and 255)
            
            b_pix = np.array(b_img)
            r_pix = np.array(r_img)
            x = np.arange(0, b_pix.shape[1])
            y = np.arange(0, b_pix.shape[0])
            
            # Get a mask to apply to the images for the roi
            b_mask = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < radius**2
            r_mask = (x[np.newaxis,:]-cx)**2 + (y[:,np.newaxis]-cy)**2 < radius**2

            # Count total number of reds, blues, and either
            r_count = np.count_nonzero(np.array(r_img)[r_mask])
            b_count = np.count_nonzero(np.array(b_img)[b_mask])
            total_count = np.count_nonzero(np.logical_or(np.array(r_img)[r_mask], np.array(b_img)[b_mask]))
            counts_dict['red'].append(r_count)
            counts_dict['blue'].append(b_count)
            counts_dict['total'].append(total_count)

            #print('r_count', r_count)
            #print('b_count', b_count)
            #print('total_count', total_count)

        df_counts = pd.DataFrame.from_dict(counts_dict)
        spheroid_blue_count = df_counts['blue'].sum()
        spheroid_red_count = df_counts['red'].sum()
        spheroid_total_count = df_counts['total'].sum()
        spheroid_total_max = df_counts['total'].max()
        spheroid_viability = 1 - (spheroid_red_count/spheroid_total_count)
        spheroid_diameter = np.sqrt((spheroid_total_max) / np.pi) * 2
        #print(spheroid_viability)
        #print(spheroid_total_count)
        #print(spheroid_total_max)
        #print(spheroid_diameter)
        # Add an entry for the spheroid; include viability, (cx, cy), diameter
        viability_dict[j] = (spheroid_viability, (cx*Pixel_to_um, cy*Pixel_to_um-204), spheroid_diameter*Pixel_to_um)
        j += 1
        
    print(len(viability_dict))
    #print(viability_dict)
    df_viability = pd.DataFrame.from_dict(viability_dict, orient='index', columns=["via", "cs", "diameter"])
    print(df_viability)
    full_df["via_" + num_str] = df_viability["via"]
    full_df["cs_" + num_str] = df_viability["cs"]
    full_df["diameter_" + num_str] = df_viability["diameter"]

    # Plotting
    cx_list = []
    cy_list = []
    via_list = []
    sArea_list = []
    for key in viability_dict:
        via, cs, diameter = viability_dict[key]
        cx, cy = cs
        cx_list.append(cx)
        cy_list.append(cy)
        via_list.append(via)
        sArea_list.append((diameter/2/6.5)**2*np.pi)

    plt.scatter(cx_list, cy_list, c=np.array(via_list)*100, s=np.array(sArea_list), cmap='RdYlBu', edgecolors='black')
    plt.clim(0,100)
    plt.colorbar()
    plt.savefig(tiff_path_1 + num_str + "circle_plot.jpg", dpi=400)
    plt.show()

full_df.to_csv(data_output_path)
end = time.time()
print("Time:", end - start)